In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_auc_score,precision_recall_curve,roc_curve
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import model_selection

from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from IPython.display import HTML

pd.set_option('max_columns', 50)

In [2]:
df = pd.read_csv('original.csv')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Index          10000 non-null  int64  
 1   Employed       10000 non-null  int64  
 2   Bank Balance   10000 non-null  float64
 3   Annual Salary  10000 non-null  float64
 4   Defaulted?     10000 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 390.8 KB


In [102]:
df.describe()

,Index,Employed,Bank Balance,Annual Salary,Defaulted?
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,0.705600,10024.498524,402203.782224,0.033300
std,2886.89568,0.455795,5804.579486,160039.674988,0.179428
min,1.00000,0.000000,0.000000,9263.640000,0.000000
25%,2500.75000,0.000000,5780.790000,256085.520000,0.000000
50%,5000.50000,1.000000,9883.620000,414631.740000,0.000000
75%,7500.25000,1.000000,13995.660000,525692.760000,0.000000
max,10000.00000,1.000000,31851.840000,882650.760000,1.000000


# Scaling of Data

In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
col=['Bank Balance', 'Annual Salary']

In [5]:
scaler = MinMaxScaler()
new_df = pd.DataFrame(scaler.fit_transform(df[col]), columns=col)

In [6]:
new_df['Employed']=df['Employed']
new_df['Defaulted?']=df['Defaulted?']
new_df.tail()

,Bank Balance,Annual Salary,Employed,Defaulted?
9995,0.268076,0.717488,1,0
9996,0.285557,0.259524,1,0
9997,0.318503,0.795032,1,0
9998,0.591116,0.493213,1,0
9999,0.075695,0.221084,0,0


# Model Selection

In [108]:
Y=df['Defaulted?']
X=df.drop(columns=['Defaulted?'],axis=1)

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,Y, test_size=0.3)

In [110]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score,precision_score,recall_score, classification_report

from time import time

In [111]:
classifiers=[
             LogisticRegression(),
             DecisionTreeClassifier(),
             RandomForestClassifier(),
             svm.SVC(),
             KNeighborsClassifier()
]

In [112]:
mse={}
for model in classifiers:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print('accuracy classification: ',accuracy_score(y_test,y_pred).round(4))
    precision_log=precision_score(y_test,y_pred).round(4)
    print("Precision: ",precision_log)
    recall_log=recall_score(y_test,y_pred).round(4)
    print("recall: ",recall_log)
    f1score=f1_score(y_test,y_pred).round(4)
    print('f1 score: ',f1score)
    print()
    print()

LogisticRegression()
	Training time: 0.036s
accuracy classification:  0.9707
Precision:  0.2
recall:  0.0244
f1 score:  0.0435


DecisionTreeClassifier()
	Training time: 0.020s
accuracy classification:  0.953
Precision:  0.2342
recall:  0.3171
f1 score:  0.2694


RandomForestClassifier()
	Training time: 0.680s
accuracy classification:  0.9753
Precision:  0.5952
recall:  0.3049
f1 score:  0.4032


SVC()
	Training time: 0.160s
accuracy classification:  0.9727
Precision:  0.0
recall:  0.0
f1 score:  0.0


KNeighborsClassifier()
	Training time: 0.012s
accuracy classification:  0.971
Precision:  0.3529
recall:  0.0732
f1 score:  0.1212




In [ ]:
from sklearn.model_selection import GridSearchCV

In [113]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [5,7,10,15],
    "min_samples_split":[0.1, 0.25, 0.5, 0.8],
    'criterion' :['gini', 'entropy']
}

In [114]:
grid_search = GridSearchCV(estimator = RandomForestClassifier(random_state=42), param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train,y_train)
grid_search.best_params_

Fitting 5 folds for each of 128 candidates, totalling 640 fits


{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'min_samples_split': 0.1,
 'n_estimators': 200}

In [115]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'min_samples_split': 0.1,
 'n_estimators': 200}

# Model Training and Prediction

In [7]:
from imblearn.over_sampling import SMOTE

In [11]:
new_df.rename(columns = {'Bank Balance': 'balance' , 'Annual Salary' : 'AnnualSalary'} ,inplace = True)

In [14]:
new_df

,balance,AnnualSalary,Employed,Defaulted?
0,0.274846,0.598905,1,0
1,0.307868,0.155727,0,0
2,0.404454,0.425862,1,0
3,0.199392,0.479959,1,0
4,0.295993,0.517867,1,0
...,...,...,...,...
9995,0.268076,0.717488,1,0
9996,0.285557,0.259524,1,0
9997,0.318503,0.795032,1,0
9998,0.591116,0.493213,1,0


In [15]:
X=new_df.drop('Defaulted?',axis=1)
Y=new_df['Defaulted?']


In [16]:
X

,balance,AnnualSalary,Employed
0,0.274846,0.598905,1
1,0.307868,0.155727,0
2,0.404454,0.425862,1
3,0.199392,0.479959,1
4,0.295993,0.517867,1
...,...,...,...
9995,0.268076,0.717488,1
9996,0.285557,0.259524,1
9997,0.318503,0.795032,1
9998,0.591116,0.493213,1


In [17]:
smote=SMOTE(sampling_strategy='minority')
X_sm, y_sm=smote.fit_resample(X,Y)

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X_sm,y_sm,test_size=0.2,random_state=15, stratify=y_sm)

In [19]:
model=RandomForestClassifier(random_state=42,criterion='gini', max_depth=10, max_features= 'auto', n_estimators= 500, min_samples_split=0.1)
model=model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [23]:
from sklearn.metrics import precision_score , recall_score , classification_report , f1_score

In [24]:
print('accuracy classification: ',accuracy_score(y_test,y_pred).round(4))
precision_log=precision_score(y_test,y_pred).round(4)
print("Precision: ",precision_log)
recall_log=recall_score(y_test,y_pred).round(4)
print("recall: ",recall_log)
f1score=f1_score(y_test,y_pred).round(4)
print('f1 score: ',f1score)

accuracy classification:  0.8782
Precision:  0.8707
recall:  0.8883
f1 score:  0.8794


In [25]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      1934
           1       0.87      0.89      0.88      1933

    accuracy                           0.88      3867
   macro avg       0.88      0.88      0.88      3867
weighted avg       0.88      0.88      0.88      3867



In [26]:
data={'accuracy score':[0.971,0.8935], 'Precision':[0.5614,0.8647],'Recall':[0.3404,0.9327], 'f1 score':[0.4238,0.8975],' Data used':['Original dataset(imbalanced)','SMOTE']}
r1=pd.DataFrame(data)
r1

,accuracy score,Precision,Recall,f1 score,Data used
0,0.9710,0.5614,0.3404,0.4238,Original dataset(imbalanced)
1,0.8935,0.8647,0.9327,0.8975,SMOTE


# Saving the model 

In [69]:
# The best parameters:  {'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}

In [34]:
import pickle 
pickle_out = open("classifier.pkl" , "wb")
pickle.dump(model , pickle_out)
pickle_out.close()

In [32]:
X_train[:2]

,balance,AnnualSalary,Employed
8475,0.000000,0.520647,1
14922,0.757842,0.866088,1


In [33]:
model.predict([[0.80000 , 0.65624 , 1]])

array([1], dtype=int64)